### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

smmSeqDataDirectory = os.path.join(getDataDir(), "Adar_smm-seq")
smmSeqBaseDataFilePath = os.path.join(smmSeqDataDirectory, "smm-seq_BPDE_all_reps.bed")

### Parse the mutation data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseCustomBed import parseCustomBed
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

if getFilesInDirectory(smmSeqDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False) is None:
    print("Parsed data not found. Generating...")
    smmSeqMutperiodInputFilePaths = parseCustomBed([smmSeqBaseDataFilePath], hg19FastaFilePath)
else:
    smmSeqMutperiodInputFilePaths = [getFilesInDirectory(smmSeqDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False)]
    print(f"Found parsed data at {smmSeqMutperiodInputFilePaths[0]}. Continuing.")

### Run the mutperiod pipeline to count mutations relative to transcription start sites.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite

TSS_FilePaths = list()
TSS_FilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_protein_coding_genes_TSSs", "hg19_protein_coding_genes_TSSs.bed"))

TSS_Names = [os.path.basename(TSS_FilePath).rsplit('.', 1)[0] for TSS_FilePath in TSS_FilePaths]

runAnalysisSuite(smmSeqMutperiodInputFilePaths, TSS_Names, normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)

In [ ]:
from mutperiodpy.CountNucleosomePositionMutations import countNucleosomePositionMutations
countNucleosomePositionMutations(smmSeqMutperiodInputFilePaths, TSS_Names, True, False, 5000, useNucStrand = True)